# Ewald Notebook

In [ ]:
%matplotlib notebook
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation
from numba import jit
import potentials
from ewald_numpy import *
from optimize import descent
from sampling import mcmc
from integrators import vv
from distances import *
from scipy.special import erfc
import line_profiler

In [ ]:
def nacl_ewald(r_cutoff, n_max, k_cutoff, max_a, step_a, E=None):
    coord =np.asarray([[0,0,0], [1/2,1/2,0], [1/2,0,1/2], [0,1/2,1/2],
                           [1/2,1/2,1/2], [1/2,0,0], [0,1/2,0],[0,0,1/2]])
    if E==None:
        charges = np.asarray([-1, -1., -1, -1, 1, 1, 1, 1])
        E = Ewald(charges, (0,1), 0.1, r_cutoff, n_max, k_cutoff)
    a_array = np.arange(0.1, max_a, step_a)
    pot_lr, pot_sr, pot_self = np.zeros_like(a_array), np.zeros_like(a_array), np.zeros_like(a_array)
    for i in range(len(a_array)):
        E.params = (a_array[i], r_cutoff, n_max, k_cutoff)
        pot_lr[i] = E.pot_lr(coord)
        pot_sr[i] = E.pot_sr(coord)
        pot_self[i] = E.pot_self()
    pot_total = pot_lr + pot_sr - pot_self
    return np.asarray([a_array, pot_lr, pot_sr, pot_self, pot_total])

In [ ]:
def madelung_formula(n):
    #taken from the wikipedia article on the Madelung constant
    x=np.arange(1, n, 2)
    arg = (x[:,None]**2+x**2)
    return 12*np.pi*np.sum(1/np.cosh(np.pi/2*np.sqrt(arg))**2)

In [ ]:
a_array, pot_lr, pot_sr, pot_self, pot_total=nacl_ewald(10, 5, 10, 5,0.1)
print("Madelung's constant from Ewald:",np.sum(pot_total[12:]/len(pot_total[12:])/8))
print("Madelung's constant from liter:",-madelung_formula(20))

In [ ]:
charges = np.asarray([-1, -1., -1, -1, 1, 1, 1, 1])
coord = np.asarray([[0,0,0], [1/2,1/2,0], [1/2,0,1/2], [0,1/2,1/2],
                           [1/2,1/2,1/2], [1/2,0,0], [0,1/2,0],[0,0,1/2]])

E = Ewald(charges, (0,1), 0.1, 0.5, 1, 1)

fig, ax = plt.subplots(1,2,figsize=(10,5))
a_array, pot_lr, pot_sr, pot_self, pot_total =nacl_ewald(10, 10, 10, 30,1)
plt1,=ax[0].plot(a_array, pot_lr, label="$\phi_{LR}$")
plt2,=ax[0].plot(a_array, pot_sr, label="$\phi_{SR}$")
plt3,=ax[0].plot(a_array, -pot_self, label="$\phi_{self}$")
plt4,=ax[0].plot(a_array, pot_total, label="$\phi_{tot}$")

circle = plt.Circle([0.5,0.5], radius=0.5, fill=False)
box = patches.Rectangle([0,0],1,1, fill=False,linestyle='--')

def update(r=2., n=1, k=1):
    a_array, pot_lr, pot_sr, pot_self, pot_total=nacl_ewald(r, n, k, 30, 1, E)
    plt1.set_ydata(pot_lr)
    plt2.set_ydata(pot_sr)
    plt4.set_ydata(pot_total)
    fig.canvas.draw()
    
    ax[1].clear()
    circle.radius = r
    ax[1].add_patch(circle)
    ax[1].add_patch(box)
    n_array = np.mgrid[-n:n+1,-n:n+1].reshape(2,-1).T
    na_coord = np.asarray([[0,1/2],[1/2,0],[1,1/2],[1/2,1]])
    na_coord = (n_array[:,None,None] + na_coord).reshape(-1,2)
    cl_coord = np.asarray([[0,0],[0,1],[1,0],[1,1],[1/2,1/2]])
    cl_coord = (n_array[:,None,None] + cl_coord).reshape(-1,2)
    ax[1].scatter(na_coord[:,0],na_coord[:,1],s=20*min(1/n,1/r))
    ax[1].scatter(cl_coord[:,0],cl_coord[:,1],s=20*min(1/n,1/r))
    fig.canvas.draw()

ax[0].set_xlabel(r'$\alpha$')
ax[0].set_title(r'Ewald potentials V splitting param. $\alpha$')
ax[0].legend()
ax[1].set_aspect('equal')
w=interact(update,
        r=widgets.FloatSlider(min=0.5,max=10,step=0.25,value=0.5,continuous_update=False),
        k=widgets.IntSlider(min=1,max=10,step=1,value=1,continuous_update=False),
        n=widgets.IntSlider(min=1,max=10,step=1,value=1,continuous_update=False));

In [ ]:
E.params = 1, 2, 1,10
charges = np.asarray([-1, -1., -1, -1, 1, 1, 1, 1])
coord = np.asarray([[0.05,0,0], [1/2,1/2,0], [1/2,0,1/2], [0,1/2,1/2],
                           [1/2,1/2,1/2], [1/2,0,0], [0,1/2,0],[0,0,1/2]])


def update(r=0.5, k=1, n=1, alpha=0.1):
    ax.clear()
    E.params = alpha, r, n, k
    force = E.force(coord)
    na_force = np.asarray([force[5],force[6]])
    cl_force = np.asarray([force[0],force[1],force[0]])
    ax.set_xlim((-1,2))
    ax.set_ylim((-1,2))
    n_array = np.mgrid[-n:n+1,-n:n+1].reshape(2,-1).T
    na_coord = np.asarray([[1/2, 0],[0,1/2]])
    cl_coord = np.asarray([[0, 0],[1/2, 1/2],[1,0]])
    na_coord = (n_array[:,None,None] + na_coord).reshape(-1,2)
    cl_coord = (n_array[:,None,None] + cl_coord).reshape(-1,2)
    na_force = (np.zeros_like(n_array)[:,None,None] + na_force[:,:2]).reshape(-1,2)
    cl_force = (np.zeros_like(n_array)[:,None,None] + cl_force[:,:2]).reshape(-1,2)
    ax.quiver(na_coord[:,0],na_coord[:,1],na_force[:,0],na_force[:,1],angles='xy')
    ax.quiver(cl_coord[:,0],cl_coord[:,1],cl_force[:,0],cl_force[:,1],angles='xy')
    ax.scatter(na_coord[:,0],na_coord[:,1])
    ax.scatter(cl_coord[:,0],cl_coord[:,1])
    fig.canvas.draw()


fig, ax = plt.subplots()
plt.xlim((-1.5,2))
plt.ylim((-1.5,2))
ax.set_aspect("equal")

w=interact(update,
        r=widgets.FloatSlider(min=1,max=10,step=0.25,value=0.5,continuous_update=False),
        k=widgets.IntSlider(min=1,max=10,step=1,value=1,continuous_update=False),
        n=widgets.IntSlider(min=1,max=10,step=1,value=1,continuous_update=False),
        alpha=widgets.FloatSlider(min=0.1, max=10, step=0.1, value=0.1, continuous_update=False));

# Optimization using gradient descent on NaCl with one of the Na+ ions being initially displaced

In [ ]:
E.params = (1, 3, 3, 5) #set Ewald parameters to ones, which were seen to be appropriate for NaCl
charges = np.asarray([-1, -1., -1, -1, 1, 1, 1, 1])
E = Ewald(charges, (0,1), 5, 3, 4, 4)
coord = np.asarray([[0.2,0,0], [1/2,1/2,0], [1/2,0,1/2], [0,1/2,1/2],
                           [1/2,1/2,1/2], [1/2,0,0], [0,1/2,0],[0,0,1/2]])#first x-coordinate is slightly displaced
n_array = np.mgrid[-2:3,-2:3,-2:3].reshape(3,-1).T
lj_pot = potentials.potentials.LJ_cut
lj_grad = potentials.gradients.LJ_cut
def grad(coord, boxsize=(0,1), pbc=True):
    L = boxsize[1] - boxsize[0]
    vecs = L*n_array[:,None,None] - vectors(coord, boxsize)
    gradient = np.zeros_like(coord)
    for v in vecs:
        gradient+=lj_grad(v, sig=1/4)
    return -E.force(coord) + gradient
def pot(coord, pbc=True,  boxsize=(0,1)):
    L = boxsize[1] - boxsize[0]
    vecs = L*n_array[:,None, None] - vectors(coord, boxsize)
    dist = distances(vecs)
    potential = 0
    for d in dist:
        potential+=np.sum(lj_pot(d, sig=1/4))
    return E.pot(coord)  + potential

In [ ]:
r_matrix, steps = descent(coord, grad, maxst=5000, boxsize=(0,1), pbc=True, save_config=True)
pot_energy = np.asarray([pot(r) for r in r_matrix])

In [ ]:
fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(121, projection='3d')
n_array2 = np.mgrid[-1:2,-1:2,-1:2].reshape(3,-1).T
colors = np.asarray(len(n_array2)*[-1, -1., -1, -1, 1, 1, 1, 1])
data = (coord + n_array2[:, None, None]).reshape(-1,3)
scat = ax.scatter(data[:,0],data[:,1], data[:,2],c=colors)
ax.set_xlim(-1,1.5)
ax.set_ylim(-0.5,2)
ax.set_zlim(-0.5,2)
ax2 = fig.add_subplot(122)
ax2.set_title("Total potential energy v step of optimization")
scat2 = ax2.scatter(0, pot_energy[0], c="red",zorder=10)
line =ax2.plot(pot_energy,zorder=5)
def animate(i):
    index = 4*i
    data = (r_matrix[index]+n_array2[:,None,None]).reshape(-1,3)
    scat._offsets3d=(data[:,0],data[:,1],data[:,2])
    scat2.set_offsets([index, pot_energy[index]])
    return scat,

#Writer = animation.writers['ffmpeg']
#writer = Writer(fps=25, bitrate=1800)
anim = animation.FuncAnimation(fig, animate, interval=50)
#anim.save('LJ_Harmonic_Particles.mp4', writer=writer)

# MCMC on NaCl with one of the Na+ ions being initially displaced with $\beta$=100

In [ ]:
r_matrix= mcmc(pot, 8, 3, 10000, 0.01, beta=100, boxsize=(0,1),pbc=True, save_config=True, init_config = None)
pot_energy = np.asarray([pot(r) for r in r_matrix])

In [ ]:
print(max(pot_energy))
fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(121, projection='3d')
n_array2 = np.mgrid[-1:2,-1:2,-1:2].reshape(3,-1).T
colors = np.asarray(len(n_array2)*[-1, -1., -1, -1, 1, 1, 1, 1])
data = (coord + n_array2[:, None, None]).reshape(-1,3)
scat = ax.scatter(data[:,0],data[:,1], data[:,2], c=colors)
ax.set_xlim(0,1)
ax.set_ylim(0,1)
ax.set_zlim(0,1)
ax2 = fig.add_subplot(122)
ax2.set_title("Total potential energy v step of optimization")
scat2 = ax2.scatter(0, pot_energy[0], c="red",zorder=10)
line =ax2.plot(pot_energy,zorder=5)
def animate(i):
    index = 4*i
    data = (r_matrix[index]+n_array2[:,None,None]).reshape(-1,3)
    scat._offsets3d=(data[:,0],data[:,1],data[:,2])
    scat2.set_offsets([index, pot_energy[index]])
    return scat,

#Writer = animation.writers['ffmpeg']
#writer = Writer(fps=25, bitrate=1800)
anim = animation.FuncAnimation(fig, animate, interval=50)
#anim.save('LJ_Harmonic_Particles.mp4', writer=writer)

# Crystal dynamics of displaced NaCl, using Velocity Verlet integrator

In [ ]:
r_matrix, v_matrix, a_matrix = vv(grad, coord, np.zeros_like(coord), 1, 1, 0.001, boxsize=(0,1), pbc=True)
pot_energy = np.asarray([pot(r) for r in r_matrix])

In [ ]:
fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(121, projection='3d')
n_array = np.mgrid[-1:2,-1:2,-1:2].reshape(3,-1).T
colors = np.asarray(len(n_array2)*[-1, -1., -1, -1, 1, 1, 1, 1])
data = (coord + n_array2[:, None, None]).reshape(-1,3)
scat = ax.scatter(data[:,0],data[:,1], data[:,2], c=colors)
ax.set_xlim(0,1)
ax.set_ylim(0,1)
ax.set_zlim(0,1)
ax2 = fig.add_subplot(122)
ax2.set_title("Total potential energy v step of optimization")
scat2 = ax2.scatter(0, pot_energy[0], c="red",zorder=10)
line =ax2.plot(pot_energy,zorder=5)
def animate(i):
    index = 4*i
    data = (r_matrix[index]+n_array2[:,None,None]).reshape(-1,3)
    scat._offsets3d=(data[:,0],data[:,1],data[:,2])
    scat2.set_offsets([index, pot_energy[index]])
    return scat,

#Writer = animation.writers['ffmpeg']
#writer = Writer(fps=25, bitrate=1800)
anim = animation.FuncAnimation(fig, animate, interval=50)
#anim.save('LJ_Harmonic_Particles.mp4', writer=writer)

In [ ]:
def ewald(E, coord, r_cutoff, n_max, k_cutoff, max_a, step_a):
    a_array = np.arange(0.1, max_a, step_a)
    pot_lr, pot_sr, pot_self = np.zeros_like(a_array), np.zeros_like(a_array), np.zeros_like(a_array)
    for i in range(len(a_array)):
        E.params = (a_array[i], r_cutoff, n_max, k_cutoff)
        pot_lr[i] = E.pot_lr(coord)
        pot_sr[i] = E.pot_sr(coord)
        pot_self[i] = E.pot_self()
    pot_total = pot_lr + pot_sr - pot_self
    return np.asarray([a_array, pot_lr, pot_sr, pot_self, pot_total])

In [ ]:
N = 10
boxsize = (0, 1)
L = boxsize[1] - boxsize[0]
coord = L*np.random.uniform(size=(N, 3))
charges = np.asarray(int(N/2)*[1,-1])

E = Ewald(charges, boxsize, 1, 5, 7, 7)
print(E.pot(coord))

def update(r=0.5, k=1, n=1):
    a_array, pot_lr, pot_sr, pot_self, pot_total=ewald(E,coord,r, n, k, 5, .1)
    plt1.set_ydata(pot_lr)
    plt2.set_ydata(pot_sr)
    plt4.set_ydata(pot_total)
    fig.canvas.draw()

fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(121)
a_array, pot_lr, pot_sr, pot_self, pot_total =ewald(E,coord,5, 2, 3, 5,.1)
plt1,=ax.plot(a_array, pot_lr, label="$\phi_{LR}$")
plt2,=ax.plot(a_array, pot_sr, label="$\phi_{SR}$")
plt3,=ax.plot(a_array, -pot_self, label="$\phi_{self}$")
plt4,=ax.plot(a_array, pot_total, label="$\phi_{tot}$")
ax.set_xlabel(r'$\alpha$')
ax.set_title(r'Ewald potentials V splitting param. $\alpha$')
ax.legend()
ax2 = fig.add_subplot(122, projection='3d')
ax2.scatter(coord[:,0],coord[:,1],coord[:,2], c=charges)
plt.show()

w=interact(update,
r=widgets.FloatSlider(min=1,max=50,step=0.25,value=5,continuous_update=False),
k=widgets.IntSlider(min=1,max=30,step=1,value=7,continuous_update=False),
n=widgets.IntSlider(min=1,max=30,step=1,value=7,continuous_update=False))